# Setup

In [1]:
import os

from dotenv import load_dotenv

from fixpoint.cache import ChatCompletionDiskTLRUCache, ChatCompletionTLRUCache
from fixpoint.utils.messages import umsg, smsg, umsg

In [9]:
from fixpoint_extras.services import formagent
from fixpoint_extras.services.formagent.setup import (
    setup_workflow
)
from fixpoint_extras.services.formagent.controllers.infogather import (
    InfoGatherer
)
from fixpoint_extras.workflows.imperative.form import Form

In [10]:
import logging
import sys
from fixpoint.logging import logger
logging.basicConfig(stream=sys.stdout)
logger.setLevel(logging.DEBUG)
logger.info("Info mode is on?")

INFO:fixpoint:Info mode is on?


In [2]:
assert load_dotenv('../.env')

In [3]:
import os
import fixpoint
from fixpoint.agents.openai import OpenAIClients
from fixpoint.cache import ChatCompletionTLRUCache

In [4]:
cache = ChatCompletionTLRUCache(maxsize=1000, ttl_s=60*60)

In [5]:
agent = fixpoint.agents.oai.OpenAI(
    model_name='gpt-3.5-turbo',
    openai_clients=OpenAIClients.from_api_key(os.environ['OPENAI_API_KEY']),
    cache=cache,
)

In [35]:
completion = agent.chat.completions.create(messages=[
    {"role": "user", "content": "What are the three laws of robotics? And what do you think of them?"},
], temperature=1)

In [36]:
print(completion.choices[0].message.content)

The Three Laws of Robotics were introduced by science fiction writer Isaac Asimov in his 1942 short story "Runaround." The laws are as follows:

1. A robot may not injure a human being or, through inaction, allow a human being to come to harm.
2. A robot must obey the orders given it by human beings except where such orders would conflict with the First Law.
3. A robot must protect its own existence as long as such protection does not conflict with the First or Second Laws.

The Three Laws of Robotics are a foundational concept in the field of robotics and artificial intelligence. They have sparked interesting conversations and debates about the ethics and moral implications of creating autonomous machines that may one day surpass humans in intelligence. Overall, the laws serve as a starting point for discussions on how to ensure the safety and well-being of humans while advancing technology. It is essential to continue to analyze and refine these laws as technology evolves to address 

In [34]:
# Types of cache modes:
#
# - skip_lookup: Don't look up keys in the cache, but write results to the
#   cache.
# - skip_all: Don't look up the cache, and don't store the result.
# - normal: Look up the cache, and store the result if it's not in the cache.
agent.fixp.set_cache_mode('skip_lookup')

completion = agent.chat.completions.create(messages=[
    {"role": "user", "content": "What are the three laws of robotics? And what do you think of them?"}
], temperature=1)

agent.fixp.set_cache_mode('normal')

In [33]:
print(completion.choices[0].message.content)

The Three Laws of Robotics were introduced by science fiction writer Isaac Asimov in his 1942 short story "Runaround." The laws are as follows:

1. A robot may not injure a human being or, through inaction, allow a human being to come to harm.
2. A robot must obey the orders given it by human beings except where such orders would conflict with the First Law.
3. A robot must protect its own existence as long as such protection does not conflict with the First or Second Laws.

The Three Laws of Robotics are a foundational concept in the field of robotics and artificial intelligence. They have sparked interesting conversations and debates about the ethics and moral implications of creating autonomous machines that may one day surpass humans in intelligence. Overall, the laws serve as a starting point for discussions on how to ensure the safety and well-being of humans while advancing technology. It is essential to continue to analyze and refine these laws as technology evolves to address 

In [5]:
cache = ChatCompletionDiskTLRUCache(
    '../localdata/diskcache/',
    ttl_s=60*60*24*7 # one week
)

In [ ]:
wfctx = setup_workflow(
    os.environ['OPENAI_API_KEY'],
    model_name='gpt-3.5-turbo',
    cache=cache,
    # uncomment this is you want to proxy requests through our logging server to log them
    # openai_base_url='http://localhost:8081/v1/proxy/openai/',
    default_openai_headers={
        "Fixpoint-Inference-API-Key": os.environ['OPENAI_API_KEY'],
        "Fixpoint-API-Key": os.environ['FIXPOINT_API_KEY'],
    }
)

In [ ]:
invoice_questions_form = Form(
    id="invoice_questions",
    path="/invoice",
    form_schema=formagent.tasks.invoice.InvoiceQuestions
)

In [ ]:
info_gatherer = InfoGatherer(invoice_questions_form, wfctx.agents['main'])

# Run the steps

## Classify

In [ ]:
# wfctx.agent.set_cache_mode('normal')
# wfctx.agent.set_cache_mode('skip_lookup')
# wfctx.cache.clear()
formtype, completion = formagent.tasks.classify_form_type(
    wfctx,
    "I am making a form to charge a client for my consulting services",
    # cache_mode="skip_lookup"
)
print()
print(completion.chain_of_thought)
print(formtype.value)

## Invoice

In [ ]:
# wfctx.agents['main'].set_cache_mode("skip_lookup")
# wfctx.agent.set_cache_mode("normal")

# inv_qs, inv_compl = formagent.tasks.answer_invoice_questions(
#     wfctx,
#     "I am sending the invoice to Dylan at dylan@fixpoint.co"
# )

invoice_info, completion = formagent.tasks.invoice.gather_invoice_info(
    wfctx,
    info_gatherer,
    user_message="I am sending the invoice to Dylan at dylan@fixpoint.co"
)

In [ ]:
invoice_info

In [ ]:
info_gatherer._questions

In [ ]:
print(info_gatherer.format_questions())

In [ ]:
invoice_info, completion = formagent.tasks.invoice.gather_invoice_info(
    wfctx,
    info_gatherer,
    # TODO(dbmikus) we should be able to answer form invoice questions based on
    # the original classification message.
    user_message="The cost is $25k, and its for my AI/LLM consulting services."
)

In [ ]:
info_gatherer.is_complete()

In [ ]:
import pprint

pprint.pprint(info_gatherer.form.contents.model_dump())
print()
pprint.pprint([h.model_dump() for h in info_gatherer.info_history])

# Scratch

In [ ]:
import json

print(json.dumps(
    formagent.tasks.invoice.InvoiceQuestions.schema(),
    indent=2
))

In [ ]:
from collections import defaultdict

# defaultdict(defaultdict(dict))
x = defaultdict(lambda: defaultdict(dict))

In [ ]:
x['w1']['t1']['s1'] = 'hi'

In [ ]:
x['w1']['t1']['s1']

In [ ]:
from fixpoint import WorkflowRun

In [ ]:
w = WorkflowRun(workflow_id="hi", id="a-run")

In [ ]:
w

In [ ]:
w.id

In [ ]:
df = wfctx.memory.to_dataframe()
df